# Training the Deepsea Network
This project incorporates wandb directly inside the YOLOv5 network (wandb support was added by ultralytics). This notebook allows us to retrain the model and see how train time augmentations can improve the networks ability to detect and track objects without increasing the data sample size. 

## Notes
Training using the ml.p3.2xlarge is about 10x faster ml.t3.medium which makes the p3 more cost effective. 

## Known Issues
When running the training script in quick succession on the g4dn.xlarge, a memory issue may arise. Shut down the running instance and wait for the machine to become available again and restart the program. This seems to be an issue related to YOLOv5 and Sagemaker's use of a RAM partition as "storage."

## Install Dependencies

In [3]:
!pip install -r deepsea-yolov5/yolov5/requirements.txt
!pip install wandb
!wandb login # append wandb login key here

  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached thop-0.1.1.post2207130030-py3-none-any.whl (15 kB)
  Using cached google_auth-2.9.1-py2.py3-none-any.whl (167 kB)
  Using cached Markdown-3.4.1-py3-none-any.whl (93 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached grpcio-1.47.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.5 MB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Using cached wandb-0.12.21-py2.py3-none-any.whl (1.8 MB)
  Using cached shortuuid-1.0.9-py3-none-any.whl (9.4 kB)
  Using cached

## Training For Evolutions
This will train the network and output the results of the generations to wandb. The best generation will be saved to data/hyps folder under the YOLOv5 directory

In [ ]:
# if training for evolutions, set epochs to 25, and evolve to 30
# if you desire to graph the evolution with the hyperparameters, add the --hyp path and change the hyperparameter.yaml file located in opt/ml/input/data/hyp.scratch-low.yaml
!export WANDB_RUN_GROUP="evolution_const_seed" && python ./deepsea-yolov5/yolov5/train.py \
--img=640 \
--data=./deepsea-yolov5/opt/ml/custom_config.yaml  \
--batch=2 \
--cfg=./deepsea-yolov5/yolov5/models/yolov5s.yaml \
--project="902005-vaa"\
--cache \
--epochs=25 \
--evolve=35

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
wandb: Currently logged in as: manuelgozzi (mbari). Use `wandb login --relogin` to force relogin
train: weights=deepsea-yolov5/yolov5/yolov5s.pt, cfg=./deepsea-yolov5/yolov5/models/yolov5s.yaml, data=./deepsea-yolov5/opt/ml/custom_config.yaml, hyp=deepsea-yolov5/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=2, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=35, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=902005-vaa, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects:

### To Train Using Current Hyperparameter File
***VERY IMPORTANT*** Before running the 50 epochs with the optimal hyperparameters, it is crucial that you copy the anchor points outputted during the evolution runs. Update the <code>yolov5*.yaml</code> file with these values and ensure that the <code>--noautoanchor</code> flag is used. The model file is located under <code>yolov5/models/</code>. 

In [4]:
# Change the contents of the deeosea-yolov5/opt/ml/input/data/hyp.scratch-low.yaml 
# to utilize the best found hyperparameters from the evolutions performed above. 
!export WANDB_RUN_GROUP="hyperparam_const_seed" && python ./deepsea-yolov5/yolov5/train.py \
--img=640 \
--data=./deepsea-yolov5/opt/ml/custom_config.yaml  \
--batch=2 \
--cfg=./deepsea-yolov5/yolov5/models/yolov5s.yaml \
--hyp=./deepsea-yolov5/opt/ml/input/data/hyp.scratch-low.yaml \
--project="902005-vaa"\
--cache \
--epochs=50

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


KeyboardInterrupt: 

## Running a sweep (doesn't work well yet)

In [ ]:
!pip install wandb

import os
os.environ['WANDB_PROJECT']="902005-vaa"

!wandb sweep deepsea-yolov5/yolov5/utils/loggers/wandb/sweep.yaml

In [ ]:
# replace mbari/.../... with output from command above
!wandb agent mbari/902005-vaa/w8krnvak